In [3]:
import cv2
import os

# Directory to save images
data_dir = 'dataset'

# Create the directory if it doesn't exist
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Function to capture images
def capture_images(person_name):
    person_dir = os.path.join(data_dir, person_name)
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)
    
    cap = cv2.VideoCapture(0)  # Adjust the camera index if necessary
    count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.imshow('Capture Images', frame)
        
        # Press 'q' to quit, 's' to save an image
        key = cv2.waitKey(1)
        if key == ord('q'):
            break
        elif key == ord('s'):
            image_path = os.path.join(person_dir, f'image{count}.jpg')
            cv2.imwrite(image_path, frame)
            print(f'Saved: {image_path}')
            count += 1
    
    cap.release()
    cv2.destroyAllWindows()

# Capture images for each person
people = ['Person1', 'Person2', 'Person3']  # Add more as needed
for person in people:
    print(f'Capture images for {person}')
    capture_images(person)

Capture images for Person1
Saved: dataset\Person1\image0.jpg
Capture images for Person2
Saved: dataset\Person2\image0.jpg
Capture images for Person3
Saved: dataset\Person3\image0.jpg


In [4]:
import os
import cv2
import numpy as np

def collect_data(data_path):
    images = []
    labels = []
    label_dict = {}
    current_label = 0
    
    for person_name in os.listdir(data_path):
        person_path = os.path.join(data_path, person_name)
        if os.path.isdir(person_path):
            label_dict[current_label] = person_name
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                image = cv2.imread(image_path)
                image = cv2.resize(image, (128, 128))
                images.append(image)
                labels.append(current_label)
            current_label += 1
    
    images = np.array(images)
    labels = np.array(labels)
    return images, labels, label_dict

data_path = 'dataset'
images, labels, label_dict = collect_data(data_path)


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_face_detection_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification (face or no face)
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

detection_model = create_face_detection_model()
detection_model.fit(images, labels, epochs=10, batch_size=32)  # Simplified training


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3333 - loss: 35.0273
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.3333 - loss: -201.5885
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.3333 - loss: -504.8044
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - accuracy: 0.3333 - loss: -963.5918
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.3333 - loss: -1617.5219
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.3333 - loss: -2523.3652
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.3333 - loss: -3735.3313
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.3333 - loss: -5347.3008
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.3333 - loss: -7430.3296
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3333 - loss: -10078.2070


In [14]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K

def triplet_loss(y_true, y_pred, alpha=0.2):
    total_lenght = y_pred.shape.as_list()[-1]
    anchor = y_pred[:, 0:int(total_lenght * 1/3)]
    positive = y_pred[:, int(total_lenght * 1/3):int(total_lenght * 2/3)]
    negative = y_pred[:, int(total_lenght * 2/3):int(total_lenght * 3/3)]
    
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    neg_dist = K.sum(K.square(anchor - negative), axis=1)
    
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0)
    return loss

def create_face_recognition_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

recognition_model = create_face_recognition_model()
recognition_model.compile(optimizer='adam', loss=triplet_loss)

# Prepare triplet data for training
def generate_triplet_data(images, labels):
    triplets = []
    for i in range(len(images)):
        anchor = images[i]
        print(images[np.where(labels == labels[i])[0][1]])
        positive = images[np.where(labels == labels[i])[0][1]]  # Another image of the same person
        negative = images[np.where(labels != labels[i])[0][0]]  # Image of a different person
        triplets.append([anchor, positive, negative])
    triplets = np.array(triplets)
    return triplets

triplet_data = generate_triplet_data(images, labels)
anchor_images = triplet_data[:, 0]
positive_images = triplet_data[:, 1]
negative_images = triplet_data[:, 2]
y_dummy = np.zeros((triplet_data.shape[0], 1))

recognition_model.fit([anchor_images, positive_images, negative_images], y_dummy, epochs=10, batch_size=32)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
import cv2
import numpy as np

def mark_attendance(name):
    with open('attendance.csv', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

def recognize_faces(frame, detection_model, recognition_model, label_dict):
    frame_resized = cv2.resize(frame, (128, 128))
    faces = detection_model.predict(np.expand_dims(frame_resized, axis=0))
    
    if faces[0] > 0.5:  # Threshold for face detection
        face_embedding = recognition_model.predict(np.expand_dims(frame_resized, axis=0))
        min_dist = float('inf')
        identity = None
        
        for label, name in label_dict.items():
            known_face_embedding = recognition_model.predict(np.expand_dims(images[np.where(labels == label)[0][0]], axis=0))
            dist = np.linalg.norm(face_embedding - known_face_embedding)
            if dist < min_dist:
                min_dist = dist
                identity = name
        
        if identity is not None:
            mark_attendance(identity)
            cv2.putText(frame, identity, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    return frame

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = recognize_faces(frame, detection_model, recognition_model, label_dict)
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()